<a href="https://colab.research.google.com/github/AlirezaSM/template-matching/blob/main/template_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
!git clone https://github.com/cplusx/QATM.git

Cloning into 'QATM'...
remote: Enumerating objects: 468, done.
remote: Total 468 (delta 0), reused 0 (delta 0), pack-reused 468
Receiving objects: 100% (468/468), 9.19 MiB | 19.60 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [4]:
!7z x drive/MyDrive/Dataset.7z -o.


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 5359681573 bytes (5112 MiB)

Extracting archive: drive/MyDrive/Dataset.7z
--
Path = drive/MyDrive/Dataset.7z
Type = 7z
Physical Size = 5359681573
Headers Size = 279463
Method = LZMA2:24
Solid = +
Blocks = 3

  0%      0% 1        0% 1 - Dataset/Sample0001_Box.txt                                     0% 18 - Dataset/Sample0006_Template.png                                           0% 23 - Dataset/Sample0008_Image.png                                        0% 36 - Dataset/Sam

# Using QATM for template matching

In [5]:
from __future__ import print_function, division

In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os, sys

In [7]:
import keras
import tensorflow as tf
from keras.layers import Input, Lambda, Concatenate
from keras import Model
import keras.backend as K
from keras.applications.vgg16 import preprocess_input
import progressbar

In [8]:
print(keras.__version__, tf.__version__)

2.9.0 2.9.2


In [9]:
from QATM.models import QATM, MyNormLayer
from QATM.utils import compute_score, all_sample_iou, plot_success_curve

In [10]:
file_dir = 'Dataset'
gt = sorted([ os.path.join(file_dir, i) for i in os.listdir(file_dir)  if '.txt' in i ])
img_path = sorted([ os.path.join(file_dir, i) for i in os.listdir(file_dir) if 'Image.png' in i ] )
tmp_path = sorted([ os.path.join(file_dir, i) for i in os.listdir(file_dir) if 'Template.png' in i ] )

gt = gt[:100]
img_path = img_path[:100]
tmp_path = tmp_path[:100]


def read_gt( file_path ):
    with open( file_path ) as IN:
        xmin, ymin, xmax, ymax = [ eval(i) for i in IN.readline().strip().split(',')]
        w = xmax - xmin
        h = ymax - ymin
    return xmin, ymin, w, h

In [11]:
import numpy as np
from scipy import signal

In [12]:
def the_corr(x):
    T_feat=x[0]
    k_feat=x[1]
    corr = np.zeros_like(T_feat)
    # for i in range(T_feat.shape[3]):
    #    corr[0,:,: ,i] = signal.correlate2d(T_feat[0 ,: ,: ,i], k_feat[0 ,: ,: ,i], boundary='symm', mode='same')
    # T_feat= T_feat *corr
    return T_feat

In [13]:
def create_model( featex, alpha = 1. ):
    T = Input( (None, None, 3), name='template_input' )
    I = Input( (None, None, 3), name='image_input' )   
    k = Input( (None, None, 3), name='key_input' )   
    
    T_feat = featex(T)
    I_feat = featex(I)
    k_feat = featex(k)


    # corr = np.zeros_like(T_feat)
    # for i in range(T_feat.shape[3]):
    #   corr[0,:,: ,i] = signal.correlate2d(T_feat[0 ,: ,: ,i], k_feat[0 ,: ,: ,i], boundary='symm', mode='same')

    # T_feat= T_feat *corr

    T_feat = keras.layers.Lambda(the_corr)([T_feat,k_feat])



    I_feat, T_feat = MyNormLayer( name='norm_layer' )( [I_feat, T_feat] )
    dist = Lambda( lambda x: tf.einsum( "xabc,xdec->xabde", K.l2_normalize(x[0], axis=-1), K.l2_normalize(x[1], axis=-1) ) , name="cosine_dist")([ I_feat, T_feat ])
    conf_map = QATM(alpha, name='qatm')(dist)
    return Model( [T, I ,k], [conf_map], name='QATM_model' )
    
def model_eval( featex, alpha=1., backup=None ):
    '''
    Have a backup featex in case image is too big.
    '''
    model = create_model( featex , alpha=alpha)
    if backup is not None:
        model_bkup = create_model( backup , alpha=alpha)
    else:
        model_bkup = model
    gt_list, gray_list, score_list = [], [], []

    num_samples = len(img_path)
    bar = progressbar.ProgressBar(max_value=num_samples)
    for idx in range(num_samples):
        bar.update(idx + 1)
        # load image and ground truth
        template = cv2.imread( tmp_path[idx] )[...,::-1]
        image = cv2.imread( img_path[idx]) [...,::-1]
        image_gt = read_gt( gt[idx] )
        x_gt, y_gt, w_gt, h_gt = [int(round(t)) for t in image_gt]
        w = image.shape[0]
        h = image.shape[1]

        #finding key point        
        candidate_size = (template.shape[0] // 2, template.shape[1] // 2)
        stride_size = (template.shape[0] // 4, template.shape[1] // 4)
        m = np.array([int(np.mean(template[...,0])), int(np.mean(template[...,1])), int(np.mean(template[...,2]))])
        start_point = [0, 0]
        min_value = 0
        min_index = [0, 0]
        for i in range(3):
            start_point[0] = i * stride_size[0]
            for j in range(3):
                start_point[1] = j * stride_size[1]
                candid = template.copy()
                candid[start_point[0]: start_point[0] + candidate_size[0], start_point[1]: start_point[1] + candidate_size[1]] = m
                i_product = sum(candid.reshape(-1)* template.reshape(-1))
                if i == 0 and j == 0:
                    min_value = i_product
                    
                if i_product < min_value:
                    min_value = i_product
                    min_index = [i, j]
        

        start_point[0] = min_index[0] * stride_size[0]
        start_point[1] = min_index[1] * stride_size[1]

        key = template.copy()
        key = key[start_point[0]: start_point[0] + candidate_size[0], start_point[1]: start_point[1] + candidate_size[1]]




        # process images
        key_= np.expand_dims(preprocess_input( key ), axis=0)
        template_ = np.expand_dims(preprocess_input( template ), axis=0)
        image_ = np.expand_dims(preprocess_input( image ) , axis=0)

        try:
          if w*h < 130000:
            val = model.predict( [template_, image_,key_ ] )
          else:
            # used when image is too big
            val = model_bkup.predict( [template_, image_,key_] )
        except:
          print(f'error index {idx}')
          break
        
        
        # compute geometry mean on score map
        val = np.log( val )
        gray = val[0,:,:,0]
        gray = cv2.resize( gray, (image.shape[1], image.shape[0]) )
        score = compute_score( gray, w_gt, h_gt )
        
        score[score>-1e-7] = -np.inf
        # plt.figure()
        # plt.imshow(score)
        # plt.show()
        gt_list.append( image_gt )
        gray_list.append( gray )
        score_list.append( score )
    return score_list, gt_list, gray_list

In [14]:
vgg19 = keras.applications.vgg19.VGG19( include_top = False, weights = 'imagenet' )

80134624/80134624 [==============================] - 3s 0us/step


In [15]:
# resize conv3_4 to conv1_2
input_ = vgg19.input
conv1_2 = vgg19.get_layer('block1_conv2').output
conv3_4 = vgg19.get_layer('block3_conv4').output
conv3_4 = Lambda( lambda x: tf.compat.v1.image.resize_bilinear( x[0], size=(tf.shape(x[1])[1], tf.shape(x[1])[2]), align_corners=True), name='resized_image' )( [conv3_4, conv1_2] )
concat = Concatenate()( [conv1_2, conv3_4] )
featex = Model( [input_], [concat], name='featex' )
# resize conv1_2 to conv3_4, used when image size is too big
input_ = vgg19.input
conv1_2 = vgg19.get_layer('block1_conv2').output
conv3_4 = vgg19.get_layer('block3_conv4').output
conv1_2 = Lambda( lambda x: tf.compat.v1.image.resize_bilinear( x[1], size=(tf.shape(x[0])[1], tf.shape(x[0])[2]), align_corners=True), name='resized_image' )( [conv3_4, conv1_2] )
concat = Concatenate()( [conv1_2, conv3_4] )
featex2 = Model( [input_], [concat], name='featex2' )

In [16]:
for k in [30]:
    score_list, gt_list, gray_list = model_eval( featex, alpha=k, backup=featex2)
    iou_score = all_sample_iou( score_list, gt_list )
    plot_success_curve( iou_score, title='alpha={} '.format(k) )

NotImplementedError: ignored